# Install packages

In [ ]:
pip install lea==3.4.4
pip install multiprocess
pip install scikit-optimize

ToDo:

# Imports

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from multisensory_playbook import detection_params_search
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor as RF
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
import pandas as pd
from multiprocess import Pool 

plt.style.use('./style_sheet.mplstyle')
from Helpers import plot_trial_data

ImportError: cannot import name 'detection_params_search' from 'multisensory_playbook' (/home/sa1523/SNN_fork/multisensory_playbook.py)

# Hyperparameters

In [5]:
p_ranges = {
    "pm": (0.0, 1.0),  # p of motion
    "pe": (0.0, 1.0),  # p of emitting given there is motion
    "pc": (0.0, 1.0),  # p correct direction when emitting
    "pn": (0.0, 1.0),  # p not neutral when not emitting
    "pi": (0.0, 0.5),  # p incorrect when emitting
}
p_labels = ["$p_m$", "$p_e$", "$p_c$", "$p_n$", "$p_i$"]

nb_trials = 1000# original: 10000
nb_steps = 90 # original: 90
search_size = 1000 # original: 10000
# probability of transitioning from e_current to e_next
trans_prob = {
    0: [0.5, 0.5],  # Probabilities for transitioning from E=0
    1: [0.5, 0.5],  # Probabilities for transitioning from E=1
}

# Optimizer setup

In [18]:
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args

def objective_function(pm, pe, pn, pc, pi, nb_trials, nb_steps, classifier_type, tasktype):
    # Generate trials
    if tasktype == 'DetectionTask':
        task = DetectionTask(**p)
        full_trials = task.generate_trials(nb_trials, nb_steps, random_seed=random_seed)
        #print("Generated trials with DetectionTask", full_trials) ### TEST
        
    elif tasktype == 'DetectionTask_Etrans_test':
        # Prepare 
        p['nb_repeats'] = nb_trials
        p['nb_steps'] = nb_steps
        p['trans_prob'] = trans_prob[0]
        task = DetectionTask_Etrans_test(**p, random_seed=random_seed) 
        full_trials = task.generate_trials
        #print("Generated trials with DetectionTask_Etrans_test", full_trials) ### TEST
        # Reset p
        keys_to_remove = {'nb_repeats', 'nb_steps', 'trans_prob'}
        p = dict(filter(lambda item: item[0] not in keys_to_remove, p.items()))
    
    # Split trials into training and testing sets for LinearClassifier training
    training_size = int(nb_trials * 0.7)  # 70% for training, adjust as needed
    testing_size = nb_trials - training_size

    # Train-test trials 
    training_trials = Trials(
        repeats=training_size,
        time_steps=nb_steps,
        M=full_trials.M[:training_size],
        A=full_trials.A[:training_size],
        V=full_trials.V[:training_size],
        task=task
    )
    testing_trials = Trials(
        repeats=testing_size,
        time_steps=nb_steps,
        M=full_trials.M[training_size:],
        A=full_trials.A[training_size:],
        V=full_trials.V[training_size:],
        task=task
    )

    # Initialize and train the classifier
    classifier = classifier_type(task)
    trained_classifier = classifier.train(training_trials)

    # Test the classifier and get accuracy
    test_results = trained_classifier.test(testing_trials)
    accuracy = test_results.accuracy

    # Return negative accuracy as we are minimizing
    return -accuracy

# Define parameter space for Bayesian Optimization
param_space = [
    Real(0.01, 0.99, "uniform", name='pm'),
    Real(0.01, 0.99, "uniform", name='pe'),
    Real(0.01, 0.99, "uniform", name='pn'),
    Real(0.01, 0.99, "uniform", name='pc'),
    Real(0.01, 0.99, "uniform", name='pi')
]


In [20]:
# Linear Optimization function
#@use_named_args(param_space)
from multisensory_playbook import LinearClassifier
from multisensory_playbook import DetectionTask
from multisensory_playbook import Trials
def objective(params):
    pm, pe, pn, pc, pi = params
    return objective_function(pm, pe, pn, pc, pi, nb_trials=100, nb_steps=10, classifier_type=LinearClassifier, tasktype=DetectionTask)

# Run Bayesian Optimization
result = gp_minimize(objective, param_space, n_calls=50, random_state=0)

# Extract best parameters and performance
best_parameters = result.x
best_performance = -result.fun

print("Best Parameters:", best_parameters)
print("Best Performance:", best_performance)

UnboundLocalError: cannot access local variable 'full_trials' where it is not associated with a value